# ワークロードを Claude 3.7 に移行する

このノートブックでは、新しい拡張思考機能の活用に重点を置き、既存のプロンプトとワークフローを Claude 3.5 Sonnet から Claude 3.7 に効果的に移行する方法を説明します。学習内容:

1. 拡張思考のメリットが得られるプロンプトを特定する方法
2. 思考の連鎖プロンプトをリファクタリングする手法
3. プロンプトを簡素化するためのベスト プラクティス
4. パフォーマンス比較による前後の例

## 前提条件
- Claude 3.5 Sonnet の機能に関する理解
- 思考の連鎖プロンプトに関する知識
- Python と Bedrock API に関する知識
- 以前のレッスン (特に拡張思考に関するレッスン 1 ～ 2) を完了していること

まず、環境を設定して必要なライブラリをインポートしましょう。

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Define model IDs for comparison
CLAUDE_35_SONNET_MODEL_ID = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)
claude_utils.verify_model_availability(bedrock, CLAUDE_35_SONNET_MODEL_ID)

## 移行のタイミングを理解する

特定の移行手法について詳しく説明する前に、どのプロンプトが Claude 3.7 の拡張思考機能への移行に適しているかを理解することが重要です。

### 移行に適した候補:
- 明示的な思考の連鎖指示を使用するプロンプト
- 1024 トークンを超える思考を必要とする複雑な推論タスク
- 複数ステップの問題解決ワークフロー
- 体系的な分析のメリットがあるタスク

### 移行にあまり適さないもの:
- 単純な事実のクエリ
- 基本的なコンテンツ生成
- 詳細な推論を必要としないタスク
- 標準プロンプトですでに適切に機能しているワークフロー

これらの違いを理解するために、具体的な例をいくつか見てみましょう。

In [ ]:
def compare_models(prompt, chain_of_thought=True, reasoning_budget=4096):
    """
    Compare responses between Claude 3.5 with chain-of-thought and Claude 3.7 with extended thinking
    
    Args:
        prompt (str): The base prompt to test
        chain_of_thought (bool): Whether to add chain-of-thought instructions for 3.5
        reasoning_budget (int): Token budget for 3.7's extended thinking
        
    Returns:
        tuple: (Claude 3.5 response, Claude 3.7 response)
    """
    # Create 3.5 prompt with optional chain-of-thought
    if chain_of_thought:
        cot_prompt = f"""
        Let's solve this step by step:
        1. First, carefully analyze what's being asked
        2. Break down the problem into parts
        3. Solve each part methodically
        4. Combine the results for a final answer

        {prompt}
        """
    else:
        cot_prompt = prompt
    
    # Get 3.5 response
    response_35 = claude_utils.invoke_claude(
        bedrock_runtime,
        cot_prompt,
        CLAUDE_35_SONNET_MODEL_ID,
        enable_reasoning=False,
        max_tokens=1000
    )
    
    # Get 3.7 response with extended thinking
    response_37 = claude_utils.invoke_claude(
        bedrock_runtime,
        prompt,  # Note: No CoT instructions needed
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=1000
    )
    
    return response_35, response_37

## 例 1: 数学の問題解決

まずは典型的な例から始めましょう。従来は思考の連鎖を促す手法を使用していた数学の文章問題です。このタイプの問題は、次のような理由から移行に最適です。
- 構造化された推論が必要
- 中間ステップを示すことでメリットが得られる
- 多くの場合、複数の計算が必要
- 中間結果の検証が必要

この問題が Claude 3.5 でどのように処理されたかと、Claude 3.7 の拡張思考でよりエレガントに解決できる方法を比較します。

In [ ]:
# Traditional math word problem
math_problem = """
A store is having a 30% off sale. A customer buys three items:
- A jacket originally priced at $120
- A pair of shoes originally priced at $85
- A shirt originally priced at $45

If there is an additional 10% discount for spending over $200 (calculated after the 30% sale discount),
how much does the customer save in total, and what is their final cost?

Show all calculations clearly.
"""

# Compare the models' responses
response_35, response_37 = compare_models(
    math_problem,
    chain_of_thought=True,  # Enable CoT for 3.5
    reasoning_budget=2048   # Moderate budget for this problem
)

# Display responses for comparison
print("Claude 3.5 Response (with chain-of-thought):")
print("-" * 80)
claude_utils.display_claude_response(response_35)

print("\nClaude 3.7 Response (with extended thinking):")
print("-" * 80)
claude_utils.display_claude_response(response_37)

### 回答の主な違い

2 つのモデルがこの問題にどのようにアプローチしたかを分析してみましょう。

1. **プロンプト構造**
- Claude 3.5 では、明示的なステップバイステップの指示が必要でした
- Claude 3.7 では、独自の推論アプローチを開発しました

2. **回答の品質**
- 計算の完全性
- 説明の明確さ
- 結果の検証

3. **トークンの効率**
- プロンプトの長さの比較
- 回答の長さの比較
- 全体的なトークンの使用

これらの違いは、数学的推論タスクにおいて、従来の思考の連鎖プロンプトよりも拡張思考の方が効果的である理由を強調しています。

In [ ]:
def display_comparison_metrics(response_35, response_37):
    """
    Display detailed comparison metrics between the two responses
    """
    # Calculate metrics for 3.5
    tokens_35 = response_35.get('usage', {}).get('totalTokens', 0)
    time_35 = response_35.get('_elapsed_time', 0)
    
    # Calculate metrics for 3.7
    tokens_37 = response_37.get('usage', {}).get('totalTokens', 0)
    time_37 = response_37.get('_elapsed_time', 0)
    
    # Display comparison
    print("Performance Comparison:")
    print("-" * 40)
    print(f"Claude 3.5 (with CoT):")
    print(f"- Total tokens: {tokens_35:,}")
    print(f"- Response time: {time_35:.2f} seconds")
    print(f"- Tokens per second: {tokens_35/time_35:.1f}")
    print("\nClaude 3.7 (with extended thinking):")
    print(f"- Total tokens: {tokens_37:,}")
    print(f"- Response time: {time_37:.2f} seconds")
    print(f"- Tokens per second: {tokens_37/time_37:.1f}")
    
    # Calculate efficiency gains/losses
    token_diff = ((tokens_37 - tokens_35) / tokens_35) * 100
    time_diff = ((time_37 - time_35) / time_35) * 100
    
    print("\nEfficiency Comparison:")
    print(f"Token usage change: {token_diff:+.1f}%")
    print(f"Response time change: {time_diff:+.1f}%")

# Display metrics for our math problem example
display_comparison_metrics(response_35, response_37)

## 例 2: 分析と推奨事項

2 番目の例では、情報を統合して推奨事項を作成する必要がある、より複雑な分析タスクを見てみましょう。このタイプのプロンプトでは、思考の連鎖がよく使用され、次のことが実現されます。
- 分析プロセスの構造化
- 複数の要因の考慮の確保
- 推奨事項の開発のガイド
- 論理的な流れの維持

この例では、Claude 3.7 の拡張思考が、規定された思考の連鎖の手順よりも自然に複雑な分析を処理できることを示します。

In [ ]:
# Complex analysis problem
analysis_prompt = """
Analyze the potential impact of implementing a four-day work week at a software company with 500 employees.

Consider:
- Employee productivity and satisfaction
- Project timelines and deadlines
- Customer support availability
- Operating costs and profitability
- Team coordination and meetings
- Industry competitiveness

Provide specific recommendations for successfully implementing this change.
"""

# Compare the models' responses
response_35, response_37 = compare_models(
    analysis_prompt,
    chain_of_thought=True,     # Enable CoT for 3.5
    reasoning_budget=4096      # Larger budget for complex analysis
)

# Display responses for comparison
print("Claude 3.5 Response (with chain-of-thought):")
print("-" * 80)
claude_utils.display_claude_response(response_35)

print("\nClaude 3.7 Response (with extended thinking):")
print("-" * 80)
claude_utils.display_claude_response(response_37)

### 移行パターン: 構造化された推論から自然な推論へ

この例は、Claude 3.5 から 3.7 への移行における重要なパターンを示しています:

1. **従来の思考の連鎖パターン**
- 明示的な分析手順が必要
- 構造的なガイダンスが必要
- 多くの場合、定型的な応答が発生

2. **拡張思考パターン**
- 自然な推論フローを開発
- 複数の視点を有機的に統合
- より微妙な分析を生成

### 移行の主な利点
- より包括的な分析
- 関連要因のより優れた統合
- より自然なアイデアの流れ
- より強力で、より文脈的な推奨事項

このパターンは、厳格な構造によって洞察の開発が制限される可能性がある複雑な分析タスクに特に役立ちます。

In [ ]:
def analyze_response_patterns(response_35, response_37):
    """
    Analyze response patterns between Claude 3.5 and 3.7
    
    Args:
        response_35 (dict): Response from Claude 3.5
        response_37 (dict): Response from Claude 3.7
        
    Returns:
        None: Prints analysis to stdout
    """
    # Safely extract responses with error handling
    text_35 = claude_utils.extract_response_content(response_35) or ""
    text_37 = claude_utils.extract_response_content(response_37) or ""
    
    if not text_35 or not text_37:
        print("Error: One or both responses are empty")
        return
    
    try:
        # Analyze structure
        sections_35 = len([l for l in text_35.split('\n') if l.strip().startswith('#')])
        sections_37 = len([l for l in text_37.split('\n') if l.strip().startswith('#')])
        
        bullets_35 = len([l for l in text_35.split('\n') if l.strip().startswith('-')])
        bullets_37 = len([l for l in text_37.split('\n') if l.strip().startswith('-')])
        
        # Calculate paragraphs safely
        paragraphs_35 = len([p for p in text_35.split('\n\n') if p.strip()])
        paragraphs_37 = len([p for p in text_37.split('\n\n') if p.strip()])
        
        # Calculate words safely
        words_35 = len(text_35.split())
        words_37 = len(text_37.split())
        
        # Display analysis
        print("Response Pattern Analysis:")
        print("-" * 40)
        print("Claude 3.5 Structure:")
        print(f"- Main sections: {sections_35}")
        print(f"- Bullet points: {bullets_35}")
        print(f"- Average words per paragraph: {words_35/max(1, paragraphs_35):.1f}")
        print(f"- Claude 3.5 v2 Response: \n\n{text_35}")
        
        print("\nClaude 3.7 Structure:")
        print(f"- Main sections: {sections_37}")
        print(f"- Bullet points: {bullets_37}")
        print(f"- Average words per paragraph: {words_37/max(1, paragraphs_37):.1f}")
        print(f"- Claude 3.7 Response: \n\n{text_37}")
        
        # Analyze differences in approach
        print("\nKey Differences:")
        print(f"- Section organization: {'More' if sections_37 > sections_35 else 'Less'} structured")
        print(f"- Point presentation: {'More' if bullets_37 > bullets_35 else 'Less'} bullet points")
        print(f"- Writing style: {'More' if words_37 > words_35 else 'Less'} detailed")
        
    except Exception as e:
        print(f"Error analyzing responses: {str(e)}")

# Analyze patterns in our analysis example
analyze_response_patterns(response_35, response_37)

## プロンプト移行のベスト プラクティス

プロンプトを Claude 3.5 から Claude 3.7 に移行する場合、移行を成功させるには、いくつかの重要なプラクティスが役立ちます。

### 1. 不要な構造を削除する
- 明示的なステップバイステップの指示を排除する
- 人工的な思考マーカーを削除する
- 拡張思考が自然な流れを生み出すようにする

### 2. トークン バジェットを調整する
- 拡張思考用に最小 1024 トークンから開始する
- タスクの複雑さに基づいてバジェットをスケールする
- 結果に基づいて監視および最適化する

### 3. 明確な要件に焦点を当てる
- 目的を明確に述べる
- 制約を直接指定する
- 関連するコンテキストを提供する

体系的な例を使用して、これらのプラクティスを調べてみましょう。

#### プロンプト移行ヘルパー:

In [ ]:
def demonstrate_prompt_migration(original_prompt, verbose=True):
    """
    Use Claude 3.7 to migrate a prompt from chain-of-thought style to extended thinking style
    
    Args:
        original_prompt (str): Original chain-of-thought style prompt
        verbose (bool): Whether to show detailed transformation steps
    
    Returns:
        tuple: (original prompt, migrated prompt)
    """
    # Create a prompt for Claude to handle the migration
    migration_request = f"""
    Help me migrate this prompt from a chain-of-thought style (used with Claude 3.5) to a cleaner style for Claude 3.7's extended thinking.

    Original prompt:
    {original_prompt}

    Guidelines for migration:
    - Remove explicit step-by-step instructions
    - Remove artificial thinking markers
    - Remove any model-specific steering language for example, instructions about laziness or verbosity
    - Remove chain-of-thought guidance and logic
    - Preserve important context and requirements
    - Make the prompt more natural and direct
    - Keep the core question/request clear
    
    Please provide only the migrated prompt with no additional explanation.
    """
    
    # Get Claude's migration suggestion
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        migration_request,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=2048,  # Moderate budget for this task
        max_tokens=1000
    )
    
    # Extract the migrated prompt
    migrated_prompt = claude_utils.extract_response_content(response)
    
    if verbose:
        print("Original Prompt:")
        print("-" * 40)
        print(original_prompt)
        print("\nMigrated Prompt:")
        print("-" * 40)
        print(migrated_prompt)
        
    return original_prompt, migrated_prompt

In [ ]:
# Example of a typical chain-of-thought prompt
original_prompt = """
Let's solve this complex optimization problem step by step:

You are planning a conference with the following constraints:
1. First, determine the venue capacity needed for 300 attendees
2. Then, calculate the budget including:
   - Venue rental ($5000/day)
   - Catering ($75/person/day)
   - Equipment rental ($2000/day)
3. Next, optimize the schedule for:
   - 3 parallel tracks
   - 45-minute sessions
   - 15-minute breaks
4. Finally, recommend the optimal conference duration in days

Think carefully about each step before providing recommendations.
"""

# Demonstrate migration
orig, migrated = demonstrate_prompt_migration(original_prompt)

# Test both versions
print("\nTesting both versions:")
response_35, response_37 = compare_models(
    migrated,  # Use migrated prompt for both to compare approaches
    chain_of_thought=True,  # 3.5 gets CoT instructions
    reasoning_budget=4096   # 3.7 gets extended thinking
)

# Show results
print("\nResults Comparison:")
display_comparison_metrics(response_35, response_37)

## 体系的な移行パターン

例に基づいて、プロンプトを Claude 3.5 から Claude 3.7 に移行する際の一般的なパターンをいくつか特定できます。

### パターン 1: ステップバイステップから目標指向へ
- **前**: 明示的な指示による詳細なステップ シーケンス
- **後**: 関連する制約とコンテキストによる明確な目標ステートメント

### パターン 2: 明示的推論から暗黙的推論へ
- **前**: 「X、Y、Z の考慮事項について考える」
- **後**: 「次の要素を考慮する: X、Y、Z」

### パターン 3: 構造化された出力から自然な出力へ
- **前**: 「セクション A、B、C で応答をフォーマットする」
- **後**: 「応答に A、B、C に関する情報を含める」

より多くの例を使用して、これらのパターンを実際に確認してみましょう。

## チェックイン ポイント: プロンプト移行の原則

プロンプトの移行について学んだことを確認してみましょう。

1. **構造の変更**
- どのような種類の指示マーカーを安全に削除できますか?
- 移行後にプロンプ​​トの長さはどのように変わりますか?
- どの要素を保持する必要がありますか?

2. **パフォーマンスへの影響**
- トークンの使用状況を比較するとどうなりますか?
- 応答時間の違いは?
- 応答品質は向上しますか?

3. **ベスト プラクティス**
- いつ構造を保持する必要がありますか?
- 適切な推論予算をどのように選択しますか?
- プロンプト移行を成功させる要因は何ですか?

これらの洞察は、独自のプロンプト移行の取り組みを導くのに役立ちます。

In [ ]:
# Define examples of different migration patterns
migration_examples = {
    "step_by_step": {
        "before": """
        Let's solve this math problem step by step:
        1. First, calculate the total cost before discount
        2. Then, apply the 15% discount
        3. Next, add 8% sales tax
        4. Finally, determine the final price
        
        How much would a $120 item cost after discount and tax?
        """,
        "complexity": "simple"
    },
    
    "explicit_reasoning": {
        "before": """
        Think through the implications of remote work by considering:
        First, analyze productivity impacts.
        Then, examine employee satisfaction effects.
        Next, evaluate communication challenges.
        Finally, weigh cost savings against potential drawbacks.
        
        Is remote work a net positive for most organizations?
        """,
        "complexity": "medium"
    },
    
    "structured_output": {
        "before": """
        Analyze this short story following these steps:
        1. First, summarize the plot
        2. Then, identify the main themes
        3. Next, analyze the character development
        4. Finally, discuss the author's writing style
        
        Structure your analysis with clear sections for each aspect.
        """,
        "complexity": "complex"
    }
}

# Migrate and display each example
for pattern_name, example in migration_examples.items():
    print(f"\n\n{'='*80}\nMigration Pattern: {pattern_name.replace('_', ' ').title()}")
    print(f"Complexity: {example['complexity'].title()}")
    
    # Migrate the prompt
    _, migrated = demonstrate_prompt_migration(example["before"], verbose=False)
    
    # Display before/after
    print("\nBefore:")
    print("-" * 40)
    print(example["before"])
    
    print("\nAfter:")
    print("-" * 40)
    print(migrated)
    
    # Determine appropriate budget based on complexity
    budget_map = {"simple": 1024, "medium": 2048, "complex": 4096}
    budget = budget_map.get(example["complexity"], 2048)
    
    print(f"\nRecommended reasoning budget: {budget} tokens")

## 実用的な移行ワークフロー

既存のプロンプトを Claude 3.7 に体系的に移行するには、次の実用的なワークフローに従ってください:

### 1. インベントリと優先順位付け
- 既存のプロンプトをすべてリストします
- 明確な思考の連鎖指示があるプロンプトを特定します
- 最もメリットのある複雑な推論タスクを優先します

### 2. 現在のプロンプトを分析
- 推論指示を特定します
- 必要なコンテキストと制約を書き留めます
- 出力形式の要件を認識します

### 3. 簡素化して移行します
- 明示的な推論手順を削除します
- 重要なコンテキストと要件を保持します
- 移行を支援するために Claude 3.7 自体の使用を検討します

### 4. テストと改良
- 最小限の推論予算 (1024 トークン) から開始します
- タスクの複雑さに基づいて必要に応じて予算を増やします
- 結果を比較し、必要に応じて調整します

この体系的なアプローチにより、Claude 3.7 の機能を最適化しながら移行を成功させることができます。

In [ ]:
def create_migration_checklist(prompt):
    """
    Generate a migration checklist for a given prompt
    
    Args:
        prompt (str): The original prompt to analyze
        
    Returns:
        dict: Migration recommendations
    """
    analysis_request = f"""
    Analyze this prompt that was designed for Claude 3.5 with chain-of-thought instructions.
    
    Prompt:
    {prompt}
    
    Please create a migration checklist with the following information in JSON format:
    1. Elements to remove (e.g., step-by-step instructions, thinking markers)
    2. Elements to preserve (e.g., key context, important constraints)
    3. Recommended reasoning budget (1024, 2048, 4096, or 8192 tokens)
    4. Complexity assessment (simple, medium, complex, very complex)
    
    Respond with only the JSON data.
    """
    
    # Get Claude's analysis
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        analysis_request,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=2048,
        max_tokens=1000
    )
    
    # Extract and parse JSON
    checklist_text = claude_utils.extract_response_content(response)
    
    try:
        # Try to parse as JSON
        import json
        checklist = json.loads(checklist_text)
        
        # Display in a readable format
        print("Migration Checklist:")
        print("-" * 40)
        print("Elements to Remove:")
        for item in checklist.get("Elements to remove", []):
            print(f"- {item}")
            
        print("\nElements to Preserve:")
        for item in checklist.get("Elements to preserve", []):
            print(f"- {item}")
            
        print(f"\nComplexity Assessment: {checklist.get('Complexity assessment', 'Unknown')}")
        print(f"Recommended Reasoning Budget: {checklist.get('Recommended reasoning budget', 1024)} tokens")
        
        return checklist
    
    except json.JSONDecodeError:
        print("Error parsing response as JSON. Raw response:")
        print(checklist_text)
        return None

# Test the migration checklist function
complex_prompt = """
Let's analyze this company's financial performance step by step:

1. First, calculate the year-over-year revenue growth rate
2. Then, analyze the profit margins (gross, operating, net)
3. Next, evaluate the debt-to-equity ratio and liquidity metrics
4. Then, compare performance against industry benchmarks
5. Finally, identify key strengths and areas for improvement

Provide a structured analysis with clear sections for each aspect.
"""

migration_checklist = create_migration_checklist(complex_prompt)

## Handling Complex Migration Scenarios

While many prompts can be easily migrated following our patterns, certain scenarios require special consideration:

### Multi-Stage Reasoning Workflows

For complex workflows that previously relied on a sequence of reasoning steps, the migration strategy needs to:
- Preserve critical dependencies between steps
- Maintain clarity about the overall goal
- Ensure all necessary context is included

### Domain-Specific Requirements

Some domains (like mathematical proofs, scientific analysis, or code generation) may have specific expectations about:
- Notation and formatting
- Methodology and approach
- Verification and validation

Let's examine how to handle these more complex migrations.

In [ ]:
# Example of a multi-stage reasoning workflow
multistage_prompt = """
Let's solve this complex data analysis problem step by step:

1. First, analyze the customer segmentation data:
   - Identify primary customer segments
   - Calculate average revenue per segment
   - Determine growth rate for each segment

2. Then, evaluate marketing channel effectiveness:
   - Compare customer acquisition cost by channel
   - Calculate ROI for each marketing channel
   - Identify the most and least effective channels

3. Next, forecast future performance:
   - Project segment growth for next 12 months
   - Estimate marketing budget requirements
   - Predict overall revenue impact

4. Finally, prioritize recommendations:
   - Rank segments by potential value
   - Suggest budget reallocation between channels
   - Propose specific strategies for high-value segments

Ensure your analysis follows this structure and provides clear recommendations.
"""

print("Complex Multi-Stage Workflow Example:")
print("-" * 60)

# Migrate the prompt
_, migrated_multistage = demonstrate_prompt_migration(multistage_prompt, verbose=False)

# Show migration results
print("\nOriginal Multi-Stage Prompt:")
print("-" * 40)
print(multistage_prompt)

print("\nMigrated Multi-Stage Prompt:")
print("-" * 40)
print(migrated_multistage)

# Generate and display migration checklist
print("\nMigration Analysis:")
migration_checklist = create_migration_checklist(multistage_prompt)

## 移行前と移行後: 全体像

移行前と移行後の完全なワークフローを比較して、学んだことをまとめてみましょう。

### Claude 3.5 のアプローチ:

1. 詳細な思考の連鎖の指示
2. 明示的なステップの順序付け
3. 構造化された推論ガイダンス
4. プロンプトでの思考プロセスの可視化

### Claude 3.7 のアプローチ:

1. 明確な目標ステートメント
2. 関連するコンテキストと制約
3. 拡張思考 (内部で処理)
4. 適切な推論予算

重要な変化は、規範的推論 (Claude に正確にどのように考えるかを伝える) から目標指向推論 (Claude に達成したいことを伝え、その拡張思考でプロセスを処理させる) への変化です。

完全なワークフローを並べて比較して、この違いを視覚化してみましょう。
![完全なワークフロー](./images/lesson8/compare.png)

## 移行前と移行後のパフォーマンス ベンチマーク

Claude 3.7 の拡張思考への移行のメリットを真に検証するには、パフォーマンスを体系的にベンチマークする必要があります。比較するためのフレームワークを作成しましょう。

1. **推論の品質**: 推論はどの程度徹底的かつ正確ですか?

2. **応答の効率**: 応答はどの程度効率的に生成されていますか?

3. **トークンの使用**: トークンの総消費量を比較するとどうですか?

さまざまな種類のプロンプトでこれらのメトリックをベンチマークすることで、データに基づいて移行の決定を下し、移行の実際の影響を理解できます。

In [ ]:
def benchmark_migration(prompts, reasoning_budgets=None):
    """
    Benchmark performance before and after migration
    
    Args:
        prompts (dict): Dictionary of prompts to benchmark
        reasoning_budgets (dict, optional): Dictionary mapping prompt names to reasoning budgets
        
    Returns:
        pd.DataFrame: Benchmark results
    """
    # Default reasoning budgets if not provided
    if reasoning_budgets is None:
        reasoning_budgets = {
            name: 4096 for name in prompts.keys()
        }
    
    results = []
    
    for name, original_prompt in prompts.items():
        print(f"\nBenchmarking: {name}")
        
        # 1. Migrate the prompt
        _, migrated_prompt = demonstrate_prompt_migration(original_prompt, verbose=False)
        
        # 2. Test Claude 3.5 with original prompt
        start_time = time.time()
        response_35 = claude_utils.invoke_claude(
            bedrock_runtime,
            original_prompt,
            CLAUDE_35_SONNET_MODEL_ID,
            enable_reasoning=False,
            max_tokens=1500
        )
        time_35 = time.time() - start_time
        
        # 3. Test Claude 3.7 with migrated prompt
        start_time = time.time()
        response_37 = claude_utils.invoke_claude(
            bedrock_runtime,
            migrated_prompt,
            CLAUDE_37_SONNET_MODEL_ID,
            enable_reasoning=True,
            reasoning_budget=reasoning_budgets.get(name, 4096),
            max_tokens=1500
        )
        time_37 = time.time() - start_time
        
        # 4. Calculate metrics
        tokens_35 = response_35.get('usage', {}).get('totalTokens', 0)
        tokens_37 = response_37.get('usage', {}).get('totalTokens', 0)
        
        # 5. Extract response text to measure length
        text_35 = claude_utils.extract_response_content(response_35) or ""
        text_37 = claude_utils.extract_response_content(response_37) or ""
        
        # 6. Store results
        results.append({
            'Prompt': name,
            'Original Length': len(original_prompt),
            'Migrated Length': len(migrated_prompt),
            'Prompt Change %': round(((len(migrated_prompt) - len(original_prompt)) / len(original_prompt) * 100), 1),
            'Claude 3.5 Tokens': tokens_35,
            'Claude 3.7 Tokens': tokens_37,
            'Token Change %': round(((tokens_37 - tokens_35) / tokens_35 * 100), 1),
            'Claude 3.5 Time (s)': round(time_35, 2),
            'Claude 3.7 Time (s)': round(time_37, 2),
            'Time Change %': round(((time_37 - time_35) / time_35 * 100), 1),
            'Claude 3.5 Output Words': len(text_35.split()),
            'Claude 3.7 Output Words': len(text_37.split()),
            'Output Change %': round(((len(text_37.split()) - len(text_35.split())) / max(1, len(text_35.split())) * 100), 1),
            'Reasoning Budget': reasoning_budgets.get(name, 4096)
        })
        
        print(f"  Original prompt: {len(original_prompt)} chars")
        print(f"  Migrated prompt: {len(migrated_prompt)} chars")
        print(f"  Claude 3.5: {tokens_35} tokens in {time_35:.2f}s")
        print(f"  Claude 3.7: {tokens_37} tokens in {time_37:.2f}s")
    
    # Create DataFrame
    df = pd.DataFrame(results)
    return df

# Define prompts for benchmarking
benchmark_prompts = {
    "Math Problem": """
    Let's solve this probability problem step by step:
    1. First, understand what we're looking for
    2. Then, identify the probability of each event
    3. Next, apply the appropriate probability formula
    4. Finally, calculate the answer
    
    In a standard deck of 52 cards, what is the probability of drawing a face card (J, Q, K) or an ace?
    """,
    
    "Business Analysis": """
    Let's analyze this business case step by step:
    1. First, identify the key issues
    2. Then, analyze the market conditions
    3. Next, evaluate the financial implications
    4. Finally, recommend a course of action
    
    A retail company is considering expanding to online sales. They have $500K to invest and want to know if they should build their own platform or use an existing marketplace like Amazon.
    """,
    
    "Code Review": """
    Let's review this code step by step:
    1. First, understand what the code is trying to do
    2. Then, check for logic errors
    3. Next, identify performance issues
    4. Finally, suggest improvements
    
    ```python
    def find_duplicates(arr):
        duplicates = []
        for i in range(len(arr)):
            for j in range(i+1, len(arr)):
                if arr[i] == arr[j] and arr[i] not in duplicates:
                    duplicates.append(arr[i])
        return duplicates
    ```
    """
}

# Define reasoning budgets for each prompt
reasoning_budgets = {
    "Math Problem": 2048,
    "Business Analysis": 4096,
    "Code Review": 4096
}

# Run the benchmark
benchmark_df = benchmark_migration(benchmark_prompts, reasoning_budgets)

# Display the results
display(HTML("<h3>Migration Benchmark Results</h3>"))
display(benchmark_df[['Prompt', 'Prompt Change %', 'Token Change %', 'Time Change %', 'Output Change %', 'Reasoning Budget']])

# Create visualizations
try:
    import matplotlib.pyplot as plt
    
    # Plot token and time changes
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    # Token changes
    benchmark_df.plot(
        x='Prompt', 
        y=['Claude 3.5 Tokens', 'Claude 3.7 Tokens'], 
        kind='bar', 
        ax=ax[0],
        color=['skyblue', 'lightgreen']
    )
    ax[0].set_title('Token Usage Comparison')
    ax[0].set_ylabel('Tokens Used')
    ax[0].tick_params(axis='x', rotation=45)
    
    # Time changes
    benchmark_df.plot(
        x='Prompt', 
        y=['Claude 3.5 Time (s)', 'Claude 3.7 Time (s)'], 
        kind='bar', 
        ax=ax[1],
        color=['skyblue', 'lightgreen']
    )
    ax[1].set_title('Response Time Comparison')
    ax[1].set_ylabel('Time (seconds)')
    ax[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Visualization could not be created: {e}")
    print("Would display charts comparing token usage and response time")

## 思考の連鎖要素を保持する場合

Claude 3.7 への移行時に思考の連鎖要素を削除することに重点を置いてきましたが、構造化されたプロンプトの特定の側面を保持することが有益な状況がいくつかあります。

### 1. 特定の出力形式要件
非常に具体的な出力形式が必要な場合は、これを明確に指定すると役立ちます。ただし、*方法* (思考プロセス) ではなく、*内容* (出力形式) に焦点を当ててください。

### 2. ドメイン固有の方法論
確立された方法論を持つ専門分野 (科学研究、法的分析など) では、必要な方法論を記載すると役立ちます。

### 3. 複数の異なるタスク
プロンプトに関連のないタスクが複数含まれている場合、これらを個別の項目として構造化すると、応答を整理するのに役立ちます (ただし、推論プロセスは整理できません)。

### 4. 非常に小さい推論予算
複雑なタスクに最小推論予算 (1024 トークン) を使用している場合は、高レベルの構造を提供すると役立つ場合があります。

重要な原則は、Claude が問題をどのように考えるべきかを規定せずに、Claude が達成すべき *内容* をガイドすることです。つまり、推論プロセスは拡張思考に任せます。

## 結論: 移行戦略チェックリスト

Claude 3.5 から Claude 3.7 へのプロンプトの移行プロセスを検討し、拡張された思考機能を活用してパフォーマンスを向上させることに重点を置いています。移行作業のガイドとなるチェックリストを以下に示します。

1. **既存のプロンプトを監査する**
- 明示的な推論指示のあるプロンプトを特定する
- 複雑な推論タスクを優先する
- 現在のトークンの使用状況とパフォーマンスを記録する

2. **プロンプトを簡素化する**
- ステップバイステップの指示を削除する
- 明示的な思考マーカーを排除する
- 重要なコンテキストと要件を保持する

3. **拡張思考を構成する**
- 最小の推論予算 (1024 トークン) から開始する
- タスクの複雑さに基づいて予算をスケールする
- 応答品質を監視し、必要に応じて調整する

4. **結果を検証する**
- 前後の応答品質を比較する
- トークンの使用状況と応答時間を測定
- フィードバックに基づいて段階的に改善する

### 重要なポイント

- 拡張思考により、より自然で柔軟な推論が可能になる
- プロンプトがシンプルになると、Claude 3.7 でより良い結果が得られることが多い
- タスクの複雑さに基づいて推論予算を選択する
- 目的 (目標) ではなく、目的に焦点を当てるそこに到達する方法 (手順)

これらのガイドラインに従うことで、既存のワークロードを正常に移行し、Claude 3.7 の高度な機能を最大限に活用できます。